# GraphModel Overview

## Intro to Knowledge Graphs

Knowledge graphs are a way of storing and organizing complex information in a graph structure. In a knowledge graph, data is represented as nodes (entities) and edges (relationships) that connect these nodes. This structure makes it easier to visualize and analyze connections between different pieces of information.

For example, in a knowledge graph representing a power grid:

* Graph _nodes_ can represent components like transformers, substations, and generators.

* Graph _edges_ can represent relationships such as the connection between different components.

It is very important to understand that in this context graph nodes are not the same as electrical buses / nodes. Every bus, branch, and piece of equipment is a node in the knowledge graph.

Below is a simple graphical representation an distribution line as a knowledge graph:

![line-property-graph](./images/4_1_property_graph.svg)

## Intro to Typed Property Graphs

The core functionality of the CIMantic Graphs interface is driven by a typed property graph interface in which the graph nodes and edges are strictly typed according to the particular object and its attributes and associations as given by UML structure of the Common Information Model. Typed property graphs are a specific kind of graph used in programming and database systems to model and manage complex data. Within the typed property graph, the following general rules apply:
1. Nodes and Edges:

    * Nodes represent entities (like transformers, substations, or generators).

    * Edges represent the relationships between these entities.

2. Properties:

    * Both nodes and edges can have properties. Properties are additional pieces of information such as names, capacity, or any other relevant attributes.

    * For example, a node representing an `ACLineSegment` will have properties like name and length.

3. Types:

    * Each node and edge has a type that defines what kind of entity or relationship it represents.

    * Types help in organizing and validating the data. For example, a node of type `ACLineSegment` might only have properties like name and length, while a node of type `PhotoVoltaicUnit` will have properties like name and rated capacity.

In the figure above, each of the graph nodes are an object with a strict datatype, such as `ACLineSegment`, `Feeder`, and `Terminal`. Only object types defined within the CIM profile selected by the user are supported within the typed property graph. The graph nodes are linked together with edges of strict typing with specific names for each association. So, the edge `ACLineSegment` --[`ConductingEquipment.Terminals`]--> `list(Terminal)` can only have a list of `Terminal` objects. A different object type, such as `Location` cannot be used in that edge.

----

## GraphModel Base Class

Most of the core interfaces for interacting with bus-branch, node-breaker, and distribution feeders models are provided by the `GraphModel` base class, which is the parent software dataclass for the classes used for specific modeling needs. The structure of the GraphModel class and its attributes are explained in more detail below. 

### Input Arguments

The following parameters need to be specified when creating a GraphModel representation of the power system:

* `connection`: An instance of `ConnectionInterface`, such as `XMLFile` or `GridappsdConnection`

* `container`: An instance of the top-level `EquipmentContainer` such as `Feeder` from which the model will be built

* `distributed`: Boolean on whether a centralized or distributed graph should be built

The `GraphModel` class itself is an abstract class and does not contain an `__init__` method. The input arguments need to be passed to a child class of `GraphModel`, such as `FeederModel`

----

### Graph

The `.graph` attribute of the GraphModel dataclass provides the interface to the typed property graph with all data contained in the network model.

It is a dictionary typed first by class type (e.g. `cim.ACLineSegment`) and then by UUID of each element within the network model.

All objects within the graph (if added using the API) are forced to have a unique UUID identifier.



----

### Object Types in the `graph`

The set of all equipment available in the graph can be obtained as the dictionary keys:

```python
eq_types = list(network.graph.keys())
```

This will provide a list of dataclass types, such as

`[cim.ACLineSegment, cim.ConnectivityNode, cim.PowerTransformer, cim.Terminal]`


----

### Object IDs in the `graph`

The set of mRIDs of all equipment contained in the graph can be obtained as the dictionary keys of the graph of a particular class type:

```python
eq_uuids = list(network.graph[cim.ACLineSegment].keys())
```

----

### Objects in the `graph`

Access to graph nodes (objects) is done via a loop or by reference to the UUID, as shown below.

#### Example 1: Access to objects via loop

The most direct method to access objects is via a loop, which covers most use cases

```python
# Find line named 634_645
for line in network.graph[cim.ACLineSegment].values():
    if line.name == '634_635':
        break
```

The .get() method can be used for more robust handling of empty fields:

```python
# Find lines with length > 100:
long_lines = []
for line in network.graph.get(cim.ACLineSegment, {}).values():
    if line.length > 100:
        long_lines.append(line)
```

#### Example 2: Access to objects via UUID

The other method is by directly invoking the mRID of the object as a UUID

```python
line = network.graph[cim.ACLineSegment][UUID('4c04f838-62aa-475e-aefa-a63b7c889c13')]
```

----

## API Reference

All classes based on GraphModel inherit the following methods:

* `add_to_graph(object)` - 

* `add_jsonld_to_graph(str)`

* `get_all_edges(cim.ClassName)`

* `get_object(mRID)`

* `get_from_triple(object,attribute)`

* `pprint(cim.ClassName)`

* `upload()`

----

### get_all_edges()

The `.get_all_edges()` method is the core library method that enables the flexibility of CIM-Graph to query for CIM objects of any class and build the knowledge graph without custom queries.

The arguments of the method are

* cim_class (type): The CIM class for which to retrieve edges (e.g. `cim.ACLineSegment`)

The method does not return any values.


----

### get_all_attributes()


The `.get_all_attributes()` method is similar to get_all_edges(), but does not create any new graph nodes. Instead, edges to new classes are represented as strings.

The arguments of the method are

* cim_class (type): The CIM class for which to retrieve edges (e.g. `cim.ACLineSegment`)

The method does not return any values.

----

### get_object()

The `get_object()` method is used to retrieve an object from the database using its mRID. 

The arguments are

* `mRID` (str): The mRID of the object to be retrieved.

* `graph` (dict[type, uuid]): Optional -- An existing graph to which the object should be added

----

### add_to_graph()

The `.add_to_graph()` method adds a CIM object to the graph model (at both the type and UUID levels within the graph dictionary).

The arguments are

* `

----

## UML Diagrams

This section contains UML class and sequence diagrams summarizing the structure and methods offered by the GraphModel class.

All diagrams are loaded from flat text using mermaid.js which can be imported using


In [1]:
from mermaid import Mermaid

### GraphModel Class Inheritance

The diagram below shows the dataclass fields and methods offered by the GraphModel base class and its children classes.

In [3]:
with open('./images/4_1_graph_model_inheritance.txt', 'r') as diagram:
    diagram_text = diagram.read()
Mermaid(diagram_text)

----

### get_all_edges()

The `.get_all_edges()` method invokes a similarly named method within the the ConnectionInterface, which performs the database-specific query. For improved processing, CIM-Graph generally uses parallel processing with sets of 100 objects queried for in each batch. The execution workflow is shown below

In [ ]:
with open('./images/4_1_get_all_edges.txt', 'r') as diagram:
    diagram_text = diagram.read()
Mermaid(diagram_text)

----

### get_all_attributes()

The get_all_attributes method invokes a similarly named method within the the ConnectionInterface, which performs the database-specific query.The execution workflow is shown below

In [4]:
with open('./images/4_1_get_all_attributes.txt', 'r') as diagram:
    diagram_text = diagram.read()
Mermaid(diagram_text)

----